In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import tifffile
from IPython.display import HTML

def create_animation_from_tiffs(input_folder, output_file=None, fps=10, max_frames=300):
    """
    Create an animation from TIFF files with filename display and frame count limit.
    """
    # Get all tiff files in the folder
    tiff_files = [f for f in sorted(os.listdir(input_folder)) if f.lower().endswith(('.tif', '.tiff'))]
    
    if not tiff_files:
        print(f"No TIFF files found in {input_folder}")
        return None
    
    all_frames = []
    frame_filenames = []  # Track filename for each frame
    
    # Process each TIFF file
    for tiff_file in tiff_files:
        # Check if we're approaching the frame limit
        if len(all_frames) >= max_frames:
            print(f"Reached maximum frame limit ({max_frames}). Stopping.")
            break
            
        file_path = os.path.join(input_folder, tiff_file)
        print(f"Processing {tiff_file}...")
        
        # Read entire image stack at once
        img_stack = tifffile.imread(file_path)
        
        # Handle based on dimensionality
        if img_stack.ndim == 3:  # Multiple frames or channels
            if img_stack.shape[2] <= 5:  # Likely channels (H,W,C)
                img_stack = np.transpose(img_stack, (2, 0, 1))  # to (C,H,W)
                # Add each channel as a separate frame with channel index
                for c in range(img_stack.shape[0]):
                    if len(all_frames) >= max_frames:
                        break
                    all_frames.append(img_stack[c])
                    frame_filenames.append(f"{tiff_file} (Ch {c+1})")
            else:  # Likely timepoints (T,H,W)
                # Add each timepoint as a separate frame
                for t in range(img_stack.shape[0]):
                    if len(all_frames) >= max_frames:
                        break
                    all_frames.append(img_stack[t])
                    frame_filenames.append(f"{tiff_file} (T {t+1})")
        else:  # 2D image
            all_frames.append(img_stack)
            frame_filenames.append(tiff_file)
    
    print(f"Total frames collected: {len(all_frames)}")
    
    # Create animation
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Determine vmin and vmax for consistent scaling
    all_data = np.concatenate([frame.flatten() for frame in all_frames])
    vmin, vmax = np.percentile(all_data, [1, 99.9])
    
    # First frame
    img_obj = ax.imshow(all_frames[0], cmap='gray', vmin=vmin, vmax=vmax)
    plt.axis('off')
    
    # Add text annotation for filename
    text_obj = ax.text(0.98, 0.98, frame_filenames[0], 
                    color='white', fontsize=12, ha='right', va='top',
                    transform=ax.transAxes,
                    bbox=dict(facecolor='black', alpha=0.7, pad=2))
    
    # Add frame counter text
    counter_obj = ax.text(0.02, 0.98, f"Frame: 1/{len(all_frames)}", 
                       color='white', fontsize=12, ha='left', va='top',
                       transform=ax.transAxes,
                       bbox=dict(facecolor='black', alpha=0.7, pad=2))
    
    plt.tight_layout()
    
    def update(frame_idx):
        img_obj.set_array(all_frames[frame_idx])
        text_obj.set_text(frame_filenames[frame_idx])
        counter_obj.set_text(f"Frame: {frame_idx+1}/{len(all_frames)}")
        return [img_obj, text_obj, counter_obj]
    
    # Create the animation
    ani = FuncAnimation(fig, update, frames=len(all_frames), blit=True)
    
    # Save the animation if output_file is provided
    if output_file:
        print(f"Saving animation to {output_file}...")
        ani.save(output_file, fps=fps, extra_args=['-vcodec', 'libx264'])
        print(f"Animation saved successfully to {output_file}")
    
    # Display in notebook
    from matplotlib import rc
    rc('animation', html='jshtml')
    return ani.to_jshtml()

In [ ]:
folder_path = "/home/nbahou/myimaging/apoDet/data/dataset2/apo_crops/Exp01_Site01"
out_path = '/home/nbahou/myimaging/apoDet/data/dataset2/movies'
display(HTML(create_animation_from_tiffs(folder_path, output_file=os.path.join(out_path, 'apo_40x_01_01.mp4'), fps=3)))